In [12]:
from cffi import FFI
import time
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import tracemalloc

In [13]:
ffibuilder = FFI()

ffibuilder.cdef("""
typedef struct node {
   int data;
   int key;
   struct node *next;
} result_t;

void *malloc(size_t size);
void free(void *ptr);
struct node * test(void);
result_t * empty(void);
void append(result_t * head);
""")

ffibuilder.set_source('_test', '''
typedef struct node {
   int data;
   int key;
   struct node *next;
} result_t;

result_t * test(void) { return calloc(1, sizeof(result_t)); }

result_t * empty(void) { 
    result_t *results;
    results = NULL;
    return results; }
    
void append(result_t * head) {
    result_t *new = calloc(1, sizeof(result_t));
    new->data = -1;
    new->key = -1;
    head->next = new;
}
''')

ffibuilder.compile(verbose=True)


generating ./_test.c
the current directory is '/Users/nams/Documents/projects/gburg-ultrasonic'
running build_ext
building '_test' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/nams/micromamba/envs/cq/include -arch x86_64 -I/Users/nams/micromamba/envs/cq/include -arch x86_64 -I/Users/nams/micromamba/envs/cq/include/python3.8 -c _test.c -o ./_test.o
gcc -bundle -undefined dynamic_lookup -L/Users/nams/micromamba/envs/cq/lib -arch x86_64 -L/Users/nams/micromamba/envs/cq/lib -arch x86_64 -arch x86_64 ./_test.o -o ./_test.cpython-38-darwin.so


'/Users/nams/Documents/projects/gburg-ultrasonic/_test.cpython-38-darwin.so'

In [14]:
from _test import ffi, lib

In [15]:
r = lib.test()
r.data = 1
r, r.data

(<cdata 'struct node *' 0x7fc640ae8c20>, 1)

In [16]:
r.next

<cdata 'struct node *' NULL>

In [17]:
r.next == ffibuilder.NULL

True

In [27]:
lib.append(r)

In [28]:
r.next.data, r.next.key, r.next.next

(-1, -1, <cdata 'struct node *' NULL>)

In [9]:
rr = r.next
lib.free(r)
lib.free(rr)

In [ ]:
rr, r.next, r

In [ ]:
rr = ffibuilder.gc(r, lib.free)

In [ ]:
ffibuilder.release(rr)

In [ ]:
lib.free(r)

In [10]:
r = lib.empty()

In [11]:
print(r)

<cdata 'struct node *' NULL>


In [ ]:
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)

In [ ]:

root = ffibuilder.new("struct node *")
link = ffibuilder.new("struct node *")
link.data = 1
root.next = link

In [ ]:
root.key, root.data

In [ ]:
root.next.key, root.next.data, root.next, root.next.next

In [ ]:
root.next.next == ffibuilder.NULL

In [ ]:
root = ffibuilder.new("result_t *")

In [ ]:
root.next

In [ ]:
lib.append(r)

In [ ]:
print(r, root)

In [ ]:
r.next = root

In [ ]:
ffi_test = FFI()
ffi_test.set_source('_test', '''
char* test(void) { return strdup("hello world"); }
''')
ffi_test.cdef('''
char* test(void);
void free(void *);
''')
ffi_test.compile(verbose=True)

In [ ]:
from _test import ffi, lib

In [ ]:
x = lib.test()
ffi.string(x)

In [ ]:
x

In [ ]:
lib.free(x)

In [ ]:
x

In [ ]:
ffi.string(x)

In [31]:
b = ffi.buffer(r)[:]

In [32]:
b, len(b)

(b'\x01\x00\x00\x00\x00\x00\x00\x00 \xfe\xf0A\xc6\x7f\x00\x00', 16)

In [25]:
r.data

1

In [29]:
bb = ffi.buffer(r.next)[:]

In [30]:
bb

b'\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00'

In [33]:
b = ffi.buffer(r)

In [34]:
len(b)

16

In [49]:
r.data = 3

In [50]:
b[:]

b'\x03\x00\x00\x00\x00\x00\x00\x00 \xfe\xf0A\xc6\x7f\x00\x00'

In [37]:
bytes(b)

b'\x01\x00\x00\x00\x00\x00\x00\x00 \xfe\xf0A\xc6\x7f\x00\x00'

In [52]:
c = ffi.buffer(r)[:]

In [55]:
c[8:]

b' \xfe\xf0A\xc6\x7f\x00\x00'

In [57]:
b'%d'%10


b'10'

In [61]:
double(1).to_bytes(4, byteorder="little")

NameError: name 'double' is not defined

In [62]:
import struct

In [149]:
dt = np.dtype([('counter', 'i4'), ('status', 'i4'), ('timestamp', 'f8')])

header = np.rec.array(np.zeros(1, dtype=dt))

header.counter = 1
header.status = 2
header.timestamp = 3.14
raw = header.tobytes()
print(header)
print(raw)

# change the counter directly in raw
raw = bytearray(raw)  # make raw mutable
raw[0] = 4
raw = bytes(raw)
print(raw)

header_frombytes = np.frombuffer(raw, dtype=dt)
print(header_frombytes)

[(1, 2, 3.14)]
b'\x01\x00\x00\x00\x02\x00\x00\x00\x1f\x85\xebQ\xb8\x1e\t@'
b'\x04\x00\x00\x00\x02\x00\x00\x00\x1f\x85\xebQ\xb8\x1e\t@'
[(4, 2, 3.14)]


In [133]:
raw = bytearray(raw)

In [136]:
raw[0] = 5

In [137]:
raw = bytes(raw)

In [138]:
raw

b'\x05\x00\x00\x00\x02\x00\x00\x00\x1f\x85\xebQ\xb8\x1e\t@'

In [111]:
h = np.rec.array(np.zeros(1, dtype=dt))

In [114]:
h.tobytes()

b'\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [126]:
h2 = np.frombuffer(h.tobytes(), dtype=dt)